In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import io
import json
import logging
import aiohttp
import os
from dataclasses import dataclass, field
from datetime import datetime, timezone
from typing import Any, List, Union
from uuid import uuid4

import pandas as pd
from redis.asyncio.client import Redis

from app.redis_transcribe import keys
from app.services.audio.audio import AudioFileCorruptedError, AudioSlicer
from app.services.audio.redis_models import (
    Meeting,
    Transcriber,
    Transcript,
    best_covering_connection,
    connection_with_minimal_start_greater_than_target,
)
from app.services.transcription.matcher import TranscriptSpeakerMatcher, TranscriptSegment, SpeakerMeta


In [4]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



2025-02-14 12:16:59,180 - INFO - Attempting Redis connection to redis:6379
2025-02-14 12:16:59,181 - INFO - Redis password is configured
2025-02-14 12:16:59,184 - INFO - Successfully connected to Redis server


In [5]:
self = processor

In [6]:
ok = True

In [8]:
while ok:
    ok = await processor.read()
    ok
    self.slice_duration
    self.max_length
    self.slice_duration/self.max_length
    processor.whisper_service_url = 'http://host.docker.internal:8033'
    await processor.transcribe()
    await processor.find_next_seek()
    await processor.do_finally()
    from app.services.audio.redis_models import Transcript
    transcript = Transcript(processor.meeting.meeting_id, processor.redis_client)
    len(await Transcript.get_all_transcripts(processor.redis_client))
    df = await Transcript.get_all_transcripts_df(processor.redis_client)
    df.reset_index(inplace=True)
    print(len(df))

2025-02-14 12:17:21,914 - ERROR - Whisper service error: 404
2025-02-14 12:17:21,915 - INFO - seek_timestamp: 2025-02-13 15:32:16+00:00


AttributeError: 'NoneType' object has no attribute 'transcriber_seek_timestamp'

In [16]:
df.tail(30)

,index,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,words,meeting_id
82,4,"Я не помню, как называется, сервис, в общем, ...",2025-02-13 15:41:23.950000+00:00,2025-02-13 15:41:34.210000+00:00,Dmitriy Grankin,1.000000,5,"[[73.71, 73.71, Я, 0.1181640625], [73.71, 73....",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
83,5,"Он как бы это некая такая волшебная, некий во...",2025-02-13 15:41:36.450000+00:00,2025-02-13 15:41:43.830000+00:00,Dmitriy Grankin,0.654472,6,"[[86.21, 86.81, Он, 0.923828125], [86.81, 87....",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
84,6,"Да-да-да, типа какая-нибудь лямбда или что-то...",2025-02-13 15:41:44.450000+00:00,2025-02-13 15:41:48.090000+00:00,Dmitriy Grankin,0.425824,7,"[[94.21, 94.65, Да, 0.98193359375], [94.65, 9...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
85,7,"Я не помню, как называется, у меня уже на нем...",2025-02-13 15:41:48.490000+00:00,2025-02-13 15:41:53.510000+00:00,Dmitriy Grankin,0.500000,8,"[[98.25, 98.85, Я, 0.99462890625], [98.85, 99...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
86,8,"Вот этот, ну, вот этот дэшборд новый, он на н...",2025-02-13 15:41:53.950000+00:00,2025-02-13 15:41:58.710000+00:00,Dmitriy Grankin,0.640756,9,"[[103.71, 103.71, Вот, 0.08294677734375], [10...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
87,9,"Он умеет также с гиппоушками работать, то ест...",2025-02-13 15:41:59.230000+00:00,2025-02-13 15:42:19.130000+00:00,Dmitriy Grankin,0.559296,10,"[[108.99, 109.33, Он, 0.92724609375], [109.33...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
88,10,"в нем внутри один тот контейнер, который самы...",2025-02-13 15:42:19.290000+00:00,2025-02-13 15:42:23.370000+00:00,Dmitriy Grankin,1.000000,11,"[[129.05, 129.73, в, 0.6650390625], [129.73, ...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
89,11,"Вот, который просто должен принять данные и з...",2025-02-13 15:42:23.950000+00:00,2025-02-13 15:42:30.950000+00:00,Dmitriy Grankin,1.000000,12,"[[133.71, 134.81, Вот,, 0.135009765625], [134...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
90,12,Ага.,2025-02-13 15:42:32.230000+00:00,2025-02-13 15:42:32.790000+00:00,Dmitriy Grankin,1.000000,13,"[[141.99, 142.55, Ага., 0.651611328125]]",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
91,13,Вот.,2025-02-13 15:42:33.450000+00:00,2025-02-13 15:42:34.010000+00:00,Dmitriy Grankin,1.000000,14,"[[143.21, 143.77, Вот., 0.9228515625]]",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
